# Multiple Inheritance, Super, and MRO TL;DR

Class initialization in Python can get tricky to understand at the beginning, especially when using multiple inheritance.

You can think that any call or field access will go to the class that got to define it last when everything was initialized. 

That order of initalization will depend on where `super().__init__(...)` is called inside each class `__init__` and also on how parent classes are ordered in the class definition.

So if you have:
```python
    class Sub(ParA, ParB, ParC):
        pass
```

and if every parent invokes super before doing anything else in `__init__`, the initialization order will be:

- ParA calls super to ParB
- ParB calls super to ParC 
- ParC calls super to object (which does nothing) and initializes itself
- ParB initializes itself
- ParA initializes itself

But lets look at some examples bellow.

First let's define two parent classes ParentA and ParentB which define `somefield_a` and `somefield_b` respectively, and for both there's the definition of the field `shared` and method `get_shared`.

We add some prints so we can see the flow.

In [44]:
class ParentA:
    def __init__(self, somefield_a :int, shared : str = None, **kwargs):
        print("ParentA calling super")
        super().__init__(**kwargs)
        print("ParentA defining methods and fields")
        self.shared = shared if shared else "ParentA" 
        self.somefield_a = somefield_a

    def a_shared(self):
        return f"ParentA is using shared from {self.shared}"
    
    def get_shared(self):
        return f"using get_shared from ParentA to get shared from {self.shared}"


class ParentB:
    def __init__(self, somefield_b :int, shared : str = None, **kwargs):
        print("ParentB calling super")
        super().__init__(**kwargs)
        print("ParentB defining methods and fields")
        self.shared = shared if shared else "ParentB" 
        self.somefield_b = somefield_b
    
    def b_shared(self):
        return f"ParentB is using shared from {self.shared}"
        
    def get_shared(self):
        return f"using get_shared from ParentB to get shared from {self.shared}"        


We also define a function `instance_report` to make it easier to print everything we need to see about the class.

In [45]:
def instance_report(instance):
    print(f"""   
    class: {type(instance)}
    instance.somefield_a: {instance.somefield_a}
    instance.somefield_b: {instance.somefield_b}
    instance.shared: {instance.shared}
    ParentA.a_shared: {instance.a_shared()}
    ParentB.b_shared: {instance.b_shared()}
    instance.get_shared: {instance.get_shared()}""")

## Calling super before anything else 

Let's look at an example subclass that inherits from both ParentA and ParentB, and also defines the field shared that exists on both parent classes.

We will start with an example that calls super before anything else (the recommended way for most use cases).

In [46]:
class SubClassSuperFirst(ParentA, ParentB):
    def __init__(self, subfield :int, shared : str = None, **kwargs):
        print("SubClassSuperFirst calling super")
        super().__init__(**kwargs)
        print("SubClassSuperFirst defining fields")
        self.shared = shared if shared else "SubClassSuperFirst" 


instance_of_SubClassSuperFirst = SubClassSuperFirst(subfield=1, somefield_a=2, somefield_b=3)

SubClassSuperFirst calling super
ParentA calling super
ParentB calling super
ParentB defining methods and fields
ParentA defining methods and fields
SubClassSuperFirst defining fields


## What is happening?

In [47]:
instance_report(instance_of_SubClassSuperFirst)

   
    class: <class '__main__.SubClassSuperFirst'>
    instance.somefield_a: 2
    instance.somefield_b: 3
    instance.shared: SubClassSuperFirst
    ParentA.a_shared: ParentA is using shared from SubClassSuperFirst
    ParentB.b_shared: ParentB is using shared from SubClassSuperFirst
    instance.get_shared: using get_shared from ParentA to get shared from SubClassSuperFirst


The way initialization works is based on two things:
- where is super being called
- how are parent classes ordered in the sub class definition

In our subclass super is called before anything else, so the last class to define the "shared" field is SubClassSuperFirst.

In this case we have ParentA and then ParentB. They both call super before anything else. So the last one that gets to define get_shared is ParentA (after calling super that initializes ParentB).


## What if we move the super call to the end on SubClassSuperFirst?

Let's look at an example subclass that inherits from both ParentA and ParentB, and also defines the field shared that exists on both parent classes.

This example calls super after defining everything else.

In [48]:
class SubClassSuperLast(ParentA, ParentB):
    def __init__(self, subfield :int, shared : str = None, **kwargs):       
        print("SubClassSuperLast defining fields")
        self.shared = shared if shared else "SubClassSuperLast" 
        self.subfield = subfield
        print("SubClassSuperLast calling super")
        super().__init__(**kwargs)

instance_of_SubClassSuperLast = SubClassSuperLast(subfield=1, somefield_a=2, somefield_b=3)

SubClassSuperLast defining fields
SubClassSuperLast calling super
ParentA calling super
ParentB calling super
ParentB defining methods and fields
ParentA defining methods and fields


In [49]:
instance_report(instance_of_SubClassSuperLast)

   
    class: <class '__main__.SubClassSuperLast'>
    instance.somefield_a: 2
    instance.somefield_b: 3
    instance.shared: ParentA
    ParentA.a_shared: ParentA is using shared from ParentA
    ParentB.b_shared: ParentB is using shared from ParentA
    instance.get_shared: using get_shared from ParentA to get shared from ParentA


Just like before we have ParentA and then ParentB. They both call super before anything else. So the last one that gets to define get_shared is ParentA (after calling super that initializes ParentB).

In this other subclass we have super called after everything else on the subclass, so the last class to define the "shared" field is ParentA.

## What if we swap ParentB and ParentA order on class definition on SubClassSuperLast?

Let's look at an example subclass that inherits from both ParentA and ParentB, and also defines the field shared that exists on both parent classes.

This example calls super after defining everything else, but swaps the order of parent classes in the class definition.

In [50]:
class SubClassSuperLastSwapped(ParentB, ParentA):
    def __init__(self, subfield :int, shared : str = None, **kwargs):       
        print("SubClassSuperLastSwapped defining fields")
        self.shared = shared if shared else "SubClassSuperLastSwapped" 
        self.subfield = subfield
        print("SubClassSuperLastSwapped calling super")
        super().__init__(**kwargs)


instance_of_SubClassSuperLastSwapped = SubClassSuperLastSwapped(subfield=1, somefield_a=2, somefield_b=3)

SubClassSuperLastSwapped defining fields
SubClassSuperLastSwapped calling super
ParentB calling super
ParentA calling super
ParentA defining methods and fields
ParentB defining methods and fields


In [51]:
instance_report(instance_of_SubClassSuperLastSwapped)

   
    class: <class '__main__.SubClassSuperLastSwapped'>
    instance.somefield_a: 2
    instance.somefield_b: 3
    instance.shared: ParentB
    ParentA.a_shared: ParentA is using shared from ParentB
    ParentB.b_shared: ParentB is using shared from ParentB
    instance.get_shared: using get_shared from ParentB to get shared from ParentB


Now instead of the previous ordering we have ParentB and then ParentA. They both call super before anything else. So the last one that gets to define get_shared is ParentB (after calling super that initializes ParentA).

Again we have super called after everything else on the subclass, so the last class to define the "shared" field is ParentB.

## What if we did the same swap of ParentB and ParentA order on class definition but on SubClassSuperFirst?

Let's look at an example subclass that inherits from both ParentA and ParentB, and also defines the field shared that exists on both parent classes.

This example calls super before defining anything else, but swaps the order of parent classes in the class definition.

In [52]:
class SubClassSuperFirstSwapped(ParentB, ParentA):
    def __init__(self, subfield :int, shared : str = None, **kwargs):
        print("SubClassSuperFirstSwapped calling super")
        super().__init__(**kwargs)
        print("SubClassSuperFirstSwapped defining fields")
        self.shared = shared if shared else "SubClassSuperFirstSwapped" 
        self.subfield = subfield


instance_of_SubClassSuperFirstSwapped = SubClassSuperFirstSwapped(subfield=1, somefield_a=2, somefield_b=3)

SubClassSuperFirstSwapped calling super
ParentB calling super
ParentA calling super
ParentA defining methods and fields
ParentB defining methods and fields
SubClassSuperFirstSwapped defining fields


In [53]:
instance_report(instance_of_SubClassSuperFirstSwapped)

   
    class: <class '__main__.SubClassSuperFirstSwapped'>
    instance.somefield_a: 2
    instance.somefield_b: 3
    instance.shared: SubClassSuperFirstSwapped
    ParentA.a_shared: ParentA is using shared from SubClassSuperFirstSwapped
    ParentB.b_shared: ParentB is using shared from SubClassSuperFirstSwapped
    instance.get_shared: using get_shared from ParentB to get shared from SubClassSuperFirstSwapped


We again have the swapped ordering with ParentB and then ParentA. They both call super before anything else. So the last one that gets to define get_shared is ParentB (after calling super that initializes ParentA).

But now we have super called after before else on the subclass, so the last class to define the "shared" field is the subclass SubClassSuperFirstSwapped.

## TL;DR 

